In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#function will take in either array of skeletons or single skeleton 
#returns: Either list of the edges or dictionary of list of the edges
def discretize_skeletons(skeletons, maximum_length, skeleton_ids=None):
        def discretize_skeleton(full_edges, maximum_length):
            p0s = full_edges[:, 0]
            p1s = full_edges[:, 1]

            diffs = p1s - p0s
            distances = np.linalg.norm(diffs, axis=1)
            inc_nums = np.ceil(distances / maximum_length).astype(int)
            inc_nums[inc_nums<2] = 2
            diffs_inc = np.repeat(diffs / inc_nums[:, None], inc_nums, axis=0)

            p0s_stack = np.repeat(p0s, inc_nums, axis=0)
            max_arange = np.arange(inc_nums.max())
            multiplicative_incrementer = np.hstack([max_arange[0:i] for i in inc_nums.tolist()])
            evenly_spaced = p0s_stack + (multiplicative_incrementer[:, None] * diffs_inc)

            total = 0
            incremented_edges = list()
            for i, p1 in zip(inc_nums, p1s):
                temp_total = total+i
                inc_edge = evenly_spaced[total:temp_total]
                inc_edge[-1] = p1
                incremented_edges.append(inc_edge)
                total = temp_total
            new_full_edges = np.vstack([np.array((inc_edge[:-1], inc_edge[1:])).transpose(1, 0, 2) for inc_edge in incremented_edges])
            return new_full_edges

        if skeleton_ids is None:
            output = list()
            for full_edges in skeletons:
                output.append(discretize_skeleton(full_edges, maximum_length))
        else:
            output = dict()
            for seg_id, full_edges in zip(skeleton_ids, skeletons):
                output[seg_id] = discretize_skeleton(full_edges, maximum_length)

        return output

In [58]:
ta3p100.CompartmentFinal.ComponentFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,n_vertex_indices,n_triangle_indices,vertex_indices preserved indices of each vertex of this sub-compartment,triangle_indices preserved indices of each triangle of this sub-compartment
2,648518346341366885,0.35,Apical,0,160393,319781,=BLOB=,=BLOB=
2,648518346341366885,0.35,Apical,1,40,73,=BLOB=,=BLOB=
2,648518346341366885,0.35,Basal,0,55172,110092,=BLOB=,=BLOB=
2,648518346341366885,0.35,Basal,1,92,172,=BLOB=,=BLOB=
2,648518346341366885,0.35,Basal,2,10,12,=BLOB=,=BLOB=
2,648518346341366885,0.35,Basal,3,276,539,=BLOB=,=BLOB=
2,648518346341366885,0.35,Error,0,9385,18694,=BLOB=,=BLOB=
2,648518346341366885,0.35,Error,1,9580,19047,=BLOB=,=BLOB=
2,648518346341366885,0.35,Error,2,79,151,=BLOB=,=BLOB=
2,648518346341366885,0.35,Oblique,0,90569,180622,=BLOB=,=BLOB=


In [46]:
key=dict(segmentation=2,segment_id=648518346341366885)
skeleton_data = (ta3p100.FilteredSkeleton() & key).fetch(as_dict=True)[0]
#skeleton_data["edges"][:, 1]
print(skeleton_data["edges"].shape)
new_skeleton = discretize_skeletons([skeleton_data["edges"]],10)[0]
np.array(new_skeleton).shape

(942, 2, 3)


(133013, 2, 3)

In [54]:
new_skeleton.reshape(-1,1,3)

array([[[286573.        , 215431.        ,  77760.8       ]],

       [[286563.52941176, 215429.98319328,  77758.0302521 ]],

       [[286563.52941176, 215429.98319328,  77758.0302521 ]],

       ...,

       [[355710.34969325, 155777.84662577,  81067.69509202]],

       [[355710.34969325, 155777.84662577,  81067.69509202]],

       [[355703.        , 155763.        ,  81056.7       ]]])

In [57]:
min(new_skeleton.reshape(-1,1,3)[:,:,0])[0]
min(new_skeleton.reshape(-1,1,3)[:,:,1])[0]
min(new_skeleton.reshape(-1,1,3)[:,:,2])[0]
max(new_skeleton.reshape(-1,1,3)[:,:,0])[0]
max(new_skeleton.reshape(-1,1,3)[:,:,1])[0]
max(new_skeleton.reshape(-1,1,3)[:,:,2])[0]

2778.58

In [22]:
#Try an example of discretizing a skeleton
ex_skeleton = (ta3p100.FilteredSkeletonMinusSoma & "segment_id=648518346341366885").fetch("edges")
print(list(ex_skeleton))
print(np.array(list(ex_skeleton)).shape)
new_discretized_skeleton = discretize_skeletons(ex_skeleton,10)
print(np.array(new_discretized_skeleton).shape)

[array([[[286573. , 215431. ,  77760.8],
        [285446. , 215310. ,  77431.2]],

       [[411610. , 261412. ,  34699.6],
        [409782. , 261948. ,  34745.5]],

       [[325932. , 196091. ,  75450.6],
        [327611. , 196889. ,  77211.1]],

       ...,

       [[347405. , 240131. ,  75622.4],
        [346373. , 240300. ,  75740.1]],

       [[315818. , 204320. ,  77069.7],
        [316132. , 206387. ,  77509.4]],

       [[356302. , 156973. ,  81952.8],
        [355703. , 155763. ,  81056.7]]])]
(1, 932, 2, 3)
(1, 130547, 2, 3)


In [29]:
np.array(ex_skeleton)[0].reshape(-1,1,3)

array([[[286573. , 215431. ,  77760.8]],

       [[285446. , 215310. ,  77431.2]],

       [[411610. , 261412. ,  34699.6]],

       ...,

       [[316132. , 206387. ,  77509.4]],

       [[356302. , 156973. ,  81952.8]],

       [[355703. , 155763. ,  81056.7]]])

In [20]:
type(new_discretized_skeleton)

list

# Conclusion don't need to discretize before hand and just do in function